# Gomoku

Student names: **Zhe HUANG, Linghao ZENG**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy
import time
import math

## Move class for Gomoku

In [3]:
class Move:
    def __init__(self, x, y, color):
        self.x = x  # Row position of the move
        self.y = y  # Column position of the move
        self.color = color  # Color of the move, 1 for White, -1 for Black

    def __str__(self):
        return f'{self.color} at ({self.x}, {self.y})'
    
    def code(self, board):
        if self.color == 1: # White
            return 2 * (board.size * self.x + self.y) + 1
        else:               # Black
            return 2 * (board.size * self.x + self.y) + 0

## Borad class for Gomoku 9x9

In [4]:
class Board:
    def __init__(self, size=9):
        self.size = size
        self.grid = [[0 for _ in range(size)] for _ in range(size)]
        self.turn = -1  # Black goes first
        self.last_move = None  # Record the last move

    def is_legal_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size and self.grid[x][y] == 0

    def check_winner(self):
        if not self.last_move: # No move has been played
            return None
        
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        x, y = self.last_move.x, self.last_move.y
        current_color = self.grid[x][y]

        for dx, dy in directions:
            count = 1
            # Check in one direction
            for i in range(1, 5):
                nx, ny = x + dx * i, y + dy * i
                if 0 <= nx < self.size and 0 <= ny < self.size and self.grid[nx][ny] == current_color:
                    count += 1
                else:
                    break
            # Check in the opposite direction
            for i in range(1, 5):
                nx, ny = x - dx * i, y - dy * i
                if 0 <= nx < self.size and 0 <= ny < self.size and self.grid[nx][ny] == current_color:
                    count += 1
                else:
                    break
            if count >= 5:
                return current_color # 1 for White, -1 for Black
            
        # Check if the board is full only after confirming there's no winner
        if all(self.grid[x][y] != 0 for x in range(self.size) for y in range(self.size)):
            return 0  # if the board is full, it's a draw

        return None
    
    def play(self, move):
        if move.color != self.turn:
            raise ValueError("Not your turn")
        if not self.is_legal_move(move.x, move.y):
            raise ValueError("Move not allowed")
        if self.check_winner() is not None:
            raise ValueError("Game has ended")
        
        self.grid[move.x][move.y] = move.color
        self.last_move = move  # Record the last move
        self.turn = -self.turn  # Switch player

    def get_legal_moves(self):
        return [Move(x, y, self.turn) for x in range(self.size) for y in range(self.size) if self.grid[x][y] == 0]

    def terminal(self):
        if self.check_winner() is not None:
            return True  # if there is a winner
        if all(self.grid[x][y] != 0 for x in range(self.size) for y in range(self.size)):
            return True  # if the board is full, it's a draw
        return False

    def playout(self):
        while not self.terminal():
            legal_moves = self.get_legal_moves()
            move = random.choice(legal_moves)
            self.play(move)
        return self.evaluate()

    def evaluate(self):
        winner = self.check_winner()
        score = None

        if winner == 1:
            score = 1    # White wins
        elif winner == -1:
            score = -1   # Black wins
        else:
            score = 0    # Draw or ongoing game

        return score
    
    def __str__(self):
        symbols = {0: '.', 1: 'W', -1: 'B'} # Empty, White, Black
        board_str = "\n".join(" ".join(symbols[cell] for cell in row) for row in self.grid)
        winner = self.check_winner()
        if winner is not None:
            winner_str = 'White' if winner == 1 else 'Black'
            return f"Winner: {winner_str}\n{board_str}"
        else:
            next_turn = 'White' if self.turn == 1 else 'Black'
            return f"Next Turn: {next_turn}\n{board_str}"


In [4]:
# Example of creating a board and applying moves
board = Board()

move = Move(7, 7, -1)  # White plays at the center
board.play(move)

move = Move(7, 8, 1)
board.play(move)

print(board)

Next Turn: Black
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . B W
. . . . . . . . .


## Flat Monte Carlo Tree Search

In [5]:
def flat(board, n):
    moves = board.get_legal_moves()
    best_score = -float('inf') # The initial best score is negative infinity
    best_move = None
    
    # Loop through all legal moves
    for move in moves:
        sum_scores = 0
        
        # Play the same move n times and average the scores
        for _ in range(n // len(moves)):
            b = copy.deepcopy(board)
            b.play(move)
            score = b.playout()
            # If it's Black's turn, we take the absolute value of the negative score
            if board.turn == -1:
                score = -score
            sum_scores += score
        
        # Update the best move and best score
        if sum_scores > best_score:
            best_score = sum_scores
            best_move = move

    return best_move, best_score


In [6]:
def execute(board, f1, f2, n=300):
    while not board.terminal():
        if board.turn == 1:  # if it's White's turn
            bestMove, _ = f1(board, n)    # White uses the first algorithm
            board.play(bestMove)
        else:                # if it's Black's turn
            bestMove, _ = f2(board, n)    # Black uses the second algorithm
            board.play(bestMove)

    return board.check_winner()

In [7]:
# flat vs flat
board = Board()
winner = execute(board, flat, flat)
print(board)

Winner: Black
B . B . B W . . .
. B B B B B . W W
. . . . . . . . W
W . W . . . . . W
. W . . . . . B .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .


In [8]:
# flat vs flat
nb_games = 100
wins = 0

# Record the time of execution
start = time.time()

# Play nb_games games and count the number of wins
for _ in range(nb_games):
    board = Board()
    winner = execute(board, flat, flat)
    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of flat(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 484.0903012752533 seconds
Winning rate of flat(Imaging we are White) vs flat: 0.35


## UCB

In [7]:
def UCB (board, n):
    """
    UCB algorithm to estimate the best possible move from a game state

    Args:
        board (Board): The game we want to extract the best move from
        n (int): Number of playouts we want to perform to estimate one best move

    Returns:
        Move: The best move to play
        float: The estimated score of the best move
    """

    moves = board.get_legal_moves()
    sumScores = [0.0 for _ in range(len(moves))]
    nbVisits = [0 for _ in range(len(moves))]
    for i in range (n):
        bestScore = -float('inf')
        bestMove = None
        for m in range (len(moves)):
            score = float('inf')
            if nbVisits [m] > 0:
                 score = sumScores[m] / nbVisits[m] + 0.4 * math.sqrt (math.log(i) / nbVisits[m])
            if score > bestScore:
                bestScore = score
                bestMove = m
        b = copy.deepcopy(board)
        b.play (moves[bestMove])
        r = b.playout()
        # If it's Black, we take the absolute value of the negative score
        if board.turn == -1:
            r = -r
        sumScores[bestMove] += r
        nbVisits[bestMove] += 1
    bestScore = 0
    bestMove = 0
    for m in range (len(moves)):
        score = nbVisits[m]
        if score > bestScore:
            bestScore = score
            bestMove = m
    return moves[bestMove], bestScore

In [10]:
# UCB vs flat
nb_games = 100
wins = 0
start = time.time()

for _ in range(nb_games):
    board = Board()
    winner = execute(board, UCB, flat)
    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of UCB(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 412.7654609680176 seconds
Winning rate of UCB(Imaging we are White) vs flat: 0.52


## Rewrite Board class for GomoKU 9x9 with hashcode

In [10]:
size = 9
hashTable = []

for k in range (3): # 3 states: empty(0), White(1), Black(-1)
    layer = []
    for i in range (size):
        row = []
        for j in range (size):
            row.append (random.randint (0, 2 ** 64))
        layer.append (row)
    hashTable.append (layer)

# Generate a random hashvalue for the current player
hashTurn = random.randint (0, 2 ** 64)

In [8]:
class Board:
    def __init__(self, size=9):
        self.size = size
        self.grid = [[0 for _ in range(size)] for _ in range(size)]
        self.turn = -1  # Black goes first
        self.last_move = None  # Record the last move
        self.h = 0

    def is_legal_move(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size and self.grid[x][y] == 0

    def check_winner(self):
        if not self.last_move: # No move has been played
            return None
        
        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        x, y = self.last_move.x, self.last_move.y
        current_color = self.grid[x][y]

        for dx, dy in directions:
            count = 1
            # Check in one direction
            for i in range(1, 5):
                nx, ny = x + dx * i, y + dy * i
                if 0 <= nx < self.size and 0 <= ny < self.size and self.grid[nx][ny] == current_color:
                    count += 1
                else:
                    break
            # Check in the opposite direction
            for i in range(1, 5):
                nx, ny = x - dx * i, y - dy * i
                if 0 <= nx < self.size and 0 <= ny < self.size and self.grid[nx][ny] == current_color:
                    count += 1
                else:
                    break
            if count >= 5:
                return current_color # 1 for White, -1 for Black
            
        # Check if the board is full only after confirming there's no winner
        if all(self.grid[x][y] != 0 for x in range(self.size) for y in range(self.size)):
            return 0  # if the board is full, it's a draw

        return None
    
    def play(self, move):
        if move.color != self.turn:
            raise ValueError("Not your turn")
        if not self.is_legal_move(move.x, move.y):
            raise ValueError("Move not allowed")
        if self.check_winner() is not None:
            raise ValueError("Game has ended")
        
        # Remove the hash value of the current move
        col = int(self.grid[move.x][move.y])
        if col != 0:
            self.h ^= hashTable[col][move.x][move.y]
        # Add the hash value of the current move
        self.h = self.h ^ hashTable[move.color][move.x][move.y] 
        self.h = self.h ^ hashTurn
            
        self.grid[move.x][move.y] = move.color
        self.last_move = move  # Record the last move
        self.turn = -self.turn  # Switch player

    def get_legal_moves(self):
        return [Move(x, y, self.turn) for x in range(self.size) for y in range(self.size) if self.grid[x][y] == 0]

    def terminal(self):
        if self.check_winner() is not None:
            return True  # if there is a winner
        if all(self.grid[x][y] != 0 for x in range(self.size) for y in range(self.size)):
            return True  # if the board is full, it's a draw
        return False

    def playout(self):
        while not self.terminal():
            legal_moves = self.get_legal_moves()
            move = random.choice(legal_moves)
            self.play(move)
        return self.evaluate()

    def evaluate(self):
        winner = self.check_winner()
        score = None

        if winner == 1:
            score = 1    # White wins
        elif winner == -1:
            score = -1   # Black wins
        else:
            score = 0    # Draw or ongoing game

        return score
    
    def __str__(self):
        symbols = {0: '.', 1: 'W', -1: 'B'} # Empty, White, Black
        board_str = "\n".join(" ".join(symbols[cell] for cell in row) for row in self.grid)
        winner = self.check_winner()
        if winner is not None:
            winner_str = 'White' if winner == 1 else 'Black'
            return f"Winner: {winner_str}\n{board_str}"
        else:
            next_turn = 'White' if self.turn == 1 else 'Black'
            return f"Next Turn: {next_turn}\n{board_str}"


## Transposition Table

In [11]:
MaxLegalMoves = size * size
Table = {}

def add(board):
    nplayouts = [0.0 for x in range(MaxLegalMoves)]
    nwins = [0.0 for x in range(MaxLegalMoves)]
    Table[board.h] = [0, nplayouts, nwins]

def look(board):
    return Table.get(board.h, None)


## UCT

In [12]:
def UCT_vs_flat(board):
    if board.terminal():
        return board.evaluate()

    if board.turn == -1: # if it's Black's turn, we use flat algorithm
        bestMove, _ = flat(board, 300)
        board.play(bestMove)
        return UCT_vs_flat(board)
    else:
        t = look(board)
        if t != None:
            bestValue = 0
            bestMove = None
            moves = board.get_legal_moves()
            for i in range(0, len(moves)):
                val = float('inf')
                n = t[0]
                ni = t[1][i] # number of playouts after move i
                wi = t[2][i] # number of wins after move i

                if ni > 0:
                    Q = wi/ni
                    val = Q + 0.4 * math.sqrt(math.log(n)/ni)

                if val > bestValue:
                    bestValue = val
                    best = i

            board.play(moves[best])
            res = UCT_vs_flat(board)
            t[0] += 1
            t[1][best] += 1
            t[2][best] += res
            return res
        else:
            add(board)
            return board.playout()
    

In [15]:
# UCT vs flat
nb_games = 100
wins = 0

# Record the time of execution
start = time.time()

# Play nb_games games and count the number of wins
for _ in range(nb_games):
    board = Board()
    winner = UCT_vs_flat(board)

    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of UCT(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 41.57500195503235 seconds
Winning rate of UCT(Imaging we are White) vs flat: 0.45


## RAVE(with AMAF)

In [13]:
def playoutAMAF (board, played): 
    while (True):
        moves = []
        moves = board.get_legal_moves()
        if len(moves) == 0 or board.terminal ():
            return board.evaluate()
        n = random.randint (0, len(moves) - 1)
        played.append(moves[n].code(board))
        board.play (moves[n])

In [14]:
MaxCodeLegalMoves = 9 * 9 * size * size * 10
Table = {}

In [15]:
def addAMAF(board):  
    nplayouts = [0.0 for x in range (MaxLegalMoves)]
    nwins = [0.0 for x in range (MaxLegalMoves)]
    nplayoutsAMAF = [0.0 for x in range (MaxCodeLegalMoves)]
    nwinsAMAF = [0.0 for x in range (MaxCodeLegalMoves)]
    Table[board.h] = [0, nplayouts, nwins, nplayoutsAMAF, nwinsAMAF]

In [16]:
def updateAMAF(t, played, res):
    for i in range(len(played)):
        if played[:i].count(played[i]) == 0:
            t[3][played[i]] += 1
            t[4][played[i]] += res

In [17]:
def RAVE_vs_flat(board, played):
    if board.terminal():
        return board.evaluate()
        
    if board.turn == -1:
        bestMove, _ = flat(board, 300)
        board.play(bestMove)
        return RAVE_vs_flat(board, played)
    else:
        t = look(board)
        if t != None:
            bestValue = 0
            best = 0
            moves = board.get_legal_moves()
            bestcode = moves[0].code(board)
            for i in range(0, len(moves)):
                val = float('inf')
                code = moves[i].code(board)
                if t[3][code] > 0:
                    beta = t[3][code] / (t[1][i] + t[3][code] + 1e-5 * t[3][code] * t[1][i])
                    Q = 1
                    if t[1][i] > 0:
                        Q = t[2][i] / t[1][i]
                    AMAF = t[4][code] / t[3][code]
                    val = beta * Q + (1 - beta) * AMAF
                if val > bestValue:
                    bestValue = val
                    best = i
                    bestcode = code
            board.play(moves[best])
            played.append(bestcode)
            res = RAVE_vs_flat(board, played)
            t[0] += 1
            t[1][best] += 1
            t[2][best] += res
            updateAMAF(t, played, res)
            return res
        else:
            addAMAF(board)
            return playoutAMAF(board, played)

In [21]:
# RAVE vs flat
nb_games = 100
wins = 0

# Record the time of execution
start = time.time()

# Play nb_games games and count the number of wins
for _ in range(nb_games):
    board = Board()
    winner = RAVE_vs_flat(board, played=[])

    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of RAVE(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 41.405999183654785 seconds
Winning rate of RAVE(Imaging we are White) vs flat: 0.52


## GRAVE

In [18]:
def GRAVE_vs_flat(board, played, tref):
    if (board.terminal()):
        return board.evaluate()

    if board.turn == -1:
        bestMove, _ = flat(board, 300)
        board.play(bestMove)
        return GRAVE_vs_flat(board, played, tref)
    else:
        t = look(board)
        if t != None:
            tr = tref
            if t[0] > 50:
                tr = t
            
            bestValue = 0
            best = 0
            moves = board.get_legal_moves()
            bestcode = moves [0].code(board)
            for i in range (0, len(moves)):
                val = float('inf')
                code = moves[i].code(board)
                if tr[3][code] > 0:
                    beta = tr[3][code] / (t[1][i] + tr [3][code] + 1e-5 * t[1][i] * tr[3][code])
                    Q = 1
                    if t[1][i] > 0:
                        Q = t[2][i] / t[1][i]
                    AMAF = tr[4][code] / tr[3][code]
                    val = (1.0 - beta) * Q + beta * AMAF
                if val > bestValue:
                    bestValue = val
                    best = i
                    bestcode = code
            board.play(moves[best])
            played.append(bestcode)
            res = GRAVE_vs_flat(board, played, tr)
            t[0] += 1
            t[1][best] += 1
            t[2][best] += res
            updateAMAF(t, played, res)
            return res
        else:
            addAMAF(board)
            return playoutAMAF(board, played)

In [23]:
# GRAVE vs flat
Table = {}
tref = []
nb_games = 100
wins = 0

# Record the time of execution
start = time.time()

# Play nb_games games and count the number of wins
for _ in range(nb_games):
    board = Board()

    nplayouts = [0.0 for x in range (MaxLegalMoves)]
    nwins = [0.0 for x in range (MaxLegalMoves)]
    nplayoutsAMAF = [0.0 for x in range (MaxCodeLegalMoves)]
    nwinsAMAF = [0.0 for x in range (MaxCodeLegalMoves)]
    tref = [0, nplayouts, nwins, nplayoutsAMAF, nwinsAMAF]

    winner = GRAVE_vs_flat(board, played=[], tref=tref)

    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of GRAVE(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 45.760936975479126 seconds
Winning rate of GRAVE(Imaging we are White) vs flat: 0.56


## Sequential Halving

In [19]:
def UCT_vs_UCT(board):
    if board.terminal():
        return board.evaluate()

    t = look(board)

    if t != None:
        bestValue = 0
        bestMove = 0
        moves = board.get_legal_moves()
        for i in range(0, len(moves)):
            val = float('inf')
            n = t[0]
            ni = t[1][i] # number of playouts after move i
            wi = t[2][i] # number of wins after move i

            if ni > 0:
                Q = wi/ni
                if board.turn == -1: # if it's Black, we take the absolute value of the negative score
                    Q  = -Q
                val = Q + 0.4 * math.sqrt(math.log(n)/ni)
            if val > bestValue:
                bestValue = val
                best = i
        board.play(moves[best])
        res = UCT_vs_UCT(board)
        t[0] += 1
        t[1][best] += 1
        t[2][best] += res
        return res
    else:
        add(board)
        return board.playout()



def sequential_halving (board, budget):
    Table = {}
    add(board)
    moves = board.get_legal_moves()
    total = len (moves)
    nplayouts = [0.0 for x in range (MaxCodeLegalMoves)]
    nwins = [0.0 for x in range (MaxCodeLegalMoves)]
    while (len (moves) > 1):
        for m in moves:
            rang = budget // (len (moves) * np.log2 (total))
            rang=rang.astype(int)
            for i in range (rang):
                b = copy.deepcopy(board)
                b.play(m)
                res = UCT_vs_UCT(b)
                nplayouts [m.code(b)] += 1
                nwins [m.code(b)] += res
                
        moves = bestHalf(board, moves, nwins, nplayouts)
    return moves[0], _


def bestHalf(board, moves, nwins, nplayouts):  
    half = []
    notused = [True for x in range (MaxCodeLegalMoves)]
    rang=np.ceil(len (moves) / 2)
    rang=rang.astype(int)
    for i in range (rang):
        best = -1.0
        bestMove = moves [0]
        for m in moves:
            code = m.code(board)
            if notused [code]:
                mu = nwins [code] / (nplayouts [code] + 1e-5)
                if mu > best:
                    best = mu
                    bestMove = m
        notused[bestMove.code(board)] = False
        half.append (bestMove)
    return half

In [25]:
board = Board()
while not board.terminal():
    if board.turn == 1:  # if it's White's turn
        bestMove, _ = sequential_halving(board, 300)    # White uses the first algorithm
        board.play(bestMove)
    else:                # if it's Black's turn
        bestMove, _ = flat(board, 300)    # Black uses the second algorithm
        board.play(bestMove)
print(board)

Winner: Black
B B B . W B W . W
. . . . B B B . W
. W W B B B . . .
. . W W B B W . .
W . . W B W . . .
. . W . B . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .


In [28]:
# Sequential Halving vs flat
nb_games = 100
wins = 0
start = time.time()

for _ in range(nb_games):
    board = Board()
    winner = execute(board, sequential_halving, flat)
    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of Sequential Halving(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 654.0044400691986 seconds
Winning rate of Sequential Halving(Imaging we are White) vs flat: 0.82


## SHUSS

In [20]:
def GRAVE_vs_GRAVE(board, played, tref):
    if (board.terminal()):
        return board.evaluate()

    t = look(board)
    if t != None:
        tr = tref
        if t[0] > 50:
            tr = t
        
        bestValue = 0
        best = 0
        moves = board.get_legal_moves()
        bestcode = moves [0].code(board)
        for i in range (0, len(moves)):
            val = float('inf')
            code = moves[i].code(board)
            if tr[3][code] > 0:
                beta = tr[3][code] / (t[1][i] + tr [3][code] + 1e-5 * t[1][i] * tr[3][code])
                Q = 1
                if t[1][i] > 0:
                    Q = t[2][i] / t[1][i]
                    if board.turn == -1:
                        Q = -Q
                AMAF = tr[4][code] / tr[3][code]
                if board.turn == -1:
                    AMAF = -AMAF
                val = (1.0 - beta) * Q + beta * AMAF
            if val > bestValue:
                bestValue = val
                best = i
                bestcode = code
        board.play(moves[best])
        played.append(bestcode)
        res = GRAVE_vs_GRAVE(board, played, tr)
        t[0] += 1
        t[1][best] += 1
        t[2][best] += res
        updateAMAF(t, played, res)
        return res
    else:
        addAMAF(board)
        return playoutAMAF(board, played)


def SHUSS(board, budget):  
    Table = {}
    addAMAF(board)
    t = look(board)
    moves = board.get_legal_moves()
    total = len (moves)
    nplayouts = [0.0 for _ in range (MaxCodeLegalMoves)]
    nwins = [0.0 for _ in range (MaxCodeLegalMoves)]
    while (len(moves) > 1):
        for m in moves:
            rang = budget // (len (moves) * np.log2(total))
            rang=rang.astype(int)
            for i in range (rang):
                b = copy.deepcopy(board)
                b.play(m)
                code = m.code(b)
                played = [code]
                res = GRAVE_vs_GRAVE(b, played, t)
                updateAMAF(t, played, res)
                nplayouts [code] += 1
                nwins [code] += res
        moves = bestHalfSHUSS(t, board, moves, nwins, nplayouts)
    return moves[0], _

def bestHalfSHUSS(t, board, moves, nwins, nplayouts):
    half = []
    notused = list(np.full(MaxCodeLegalMoves, True))
    c = 128
    rang=np.ceil(len (moves) / 2)
    rang=rang.astype(int)
    for i in range (rang):
        best = -1.0
        bestMove = moves [0]
        for m in moves:
            code = m.code(board)
            if notused [code]:
                AMAF = t[4][code] / (t[3][code] + 1e-5)
                mu = nwins [code] / (nplayouts[code] + 1e-5) + c * AMAF / (nplayouts[code] + 1e-5)
                if mu > best:
                    best = mu
                    bestMove = m
        notused[bestMove.code(board)] = False
        half.append (bestMove)
    return half

In [38]:
board = Board()
while not board.terminal():
    if board.turn == 1:  # if it's White's turn
        bestMove, _ = SHUSS(board, 300)    # White uses the first algorithm
        board.play(bestMove)
    else:                # if it's Black's turn
        bestMove, _ = flat(board, 300)    # Black uses the second algorithm
        board.play(bestMove)
print(board)

Winner: White
. . . B . B B W .
. . . B . . . . .
. W . . B . . . B
. . B . W . . . .
. W W W W W . . .
. . . . . . . . B
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .


In [20]:
# SHUSS vs flat
nb_games = 100
wins = 0
start = time.time()

for _ in range(nb_games):
    board = Board()
    winner = execute(board, SHUSS, flat)
    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of SHUSS(Imaging we are White) vs flat: {wins/nb_games}")

: 

# Misere GomoKU

In [21]:
class Misere_Board(Board):
    """
    In a misère game, the longer the play lasts, the lower the score becomes.
    """

    def discountedPlayout(self):
        t = 0
        while not self.terminal():
            legal_moves = self.get_legal_moves()
            move = random.choice(legal_moves)
            self.play(move)
            t += 1
        
        return self.evaluate() / (t + 1)

## Nested Flat

In [24]:
def Nested_flat(board, n):
    moves = board.get_legal_moves()
    best_score = -float('inf') # The initial best score is negative infinity
    best_move = None
    
    # Loop through all legal moves
    for move in moves:
        sum_scores = 0
        
        # Play the same move n times and average the scores
        for _ in range(n // len(moves)):
            b = copy.deepcopy(board)
            b.play(move)
            score = b.discountedPlayout()
            # If it's Black's turn, we take the absolute value of the negative score
            if board.turn == -1:
                score = -score
            sum_scores += score
        
        # Update the best move and best score
        if sum_scores > best_score:
            best_score = sum_scores
            best_move = move

    return best_move, best_score

In [25]:
# Nested_flat vs flat
nb_games = 100
wins = 0
start = time.time()

for _ in range(nb_games):
    board = Misere_Board()

    winner = execute(board, Nested_flat, flat)

    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of Nested_flat (Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 560.7055070400238 seconds
Winning rate of Nested_flat (Imaging we are White) vs flat: 0.59


## Nested UCT

In [33]:
def Nested_UCT_vs_flat(board, t1):
    if board.terminal():
        return board.evaluate() / (t1+1)

    if board.turn == -1: # if it's Black's turn, we use flat algorithm
        bestMove, _ = flat(board, 300)
        board.play(bestMove)
        t1 += 1
        return Nested_UCT_vs_flat(board, t1)
    else:
        t = look(board)
        if t != None:
            bestValue = 0
            bestMove = None
            moves = board.get_legal_moves()
            for i in range(0, len(moves)):
                val = float('inf')
                n = t[0]
                ni = t[1][i] # number of playouts after move i
                wi = t[2][i] # number of wins after move i

                if ni > 0:
                    Q = wi/ni
                    val = Q + 0.4 * math.sqrt(math.log(n)/ni)

                if val > bestValue:
                    bestValue = val
                    best = i

            board.play(moves[best])
            t1 += 1
            res = Nested_UCT_vs_flat(board, t1)
            t[0] += 1
            t[1][best] += 1
            t[2][best] += res
            return res
        else:
            add(board)
            return board.discountedPlayout()
    

In [37]:
# Nested_UCT vs flat in Misere Gomoku
nb_games = 100
wins = 0

# Record the time of execution
start = time.time()

# Play nb_games games and count the number of wins
for _ in range(nb_games):
    board = Misere_Board()
    Nested_UCT_vs_flat(board, 0)

    winner = board.check_winner()
    if winner == 1:
        wins += 1

end = time.time()

print(f"Time taken: {end - start} seconds")
print(f"Winning rate of Nested_UCT(Imaging we are White) vs flat: {wins/nb_games}")

Time taken: 69.67531204223633 seconds
Winning rate of Nested_UCT(Imaging we are White) vs flat: 0.46
